In [ ]:
# Instalace závislostí:
# pip install neo4j pygame ipywidgets mutagen

import pygame
import os
from neo4j import GraphDatabase
from IPython.display import display, clear_output
import ipywidgets as widgets
from pathlib import Path
import hashlib
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
import time

# Potlačení varování pkgrecources
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pkg_resources')

In [ ]:
# === 1. NEO4J PŘIPOJENÍ ===
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record.data() for record in result]

# Globální připojení
neo4j_conn = Neo4jConnection(
    uri="neo4j://127.0.0.1:7687",
    user="neo4j",
    password="heslo123"  # Změň na své heslo
)

In [ ]:
# === 2. SPRÁVA UŽIVATELŮ ===
class UserManager:
    def __init__(self, neo4j_conn):
        self.conn = neo4j_conn

    # Registrace nového uživatele
    def register_user(self, username):
        # Kontrola, zda uživatel již existuje
        check_query = """
        MATCH (u:User {name: $username})
        RETURN u.userId as userId
        """
        result = self.conn.query(check_query, {"username": username})

        if result:
            return None, "Uživatelské jméno již existuje"

        # Vytvoření unikátního userId
        user_id = hashlib.md5(username.encode()).hexdigest()[:8]

        # Vytvoření uzlu User
        create_query = """
        CREATE (u:User {name: $username, userId: $userId})
        RETURN u.userId as userId
        """
        self.conn.query(create_query,
                        {"username": username, "userId": user_id})

        return user_id, "Registrace úspěšná"

    # Přihlášení existujícího uživatele
    def login_user(self, username):
        query = """
        MATCH (u:User {name: $username})
        RETURN u.userId as userId, u.name as name
        """
        result = self.conn.query(query, {"username": username})

        if result:
            return result[0]['userId'], "Přihlášení úspěšné"
        else:
            return None, "Uživatel neexistuje"

In [ ]:
# === 3. NAČÍTÁNÍ MP3 SOUBORŮ ===
class MusicLibraryScanner:
    def __init__(self, neo4j_conn):
        self.conn = neo4j_conn

    def scan_directory(self, directory_path):
        """Naskenuje složku a vytvoří uzly v Neo4j"""
        mp3_files = list(Path(directory_path).rglob("*.mp3"))

        if not mp3_files:
            print(f"✗ Žádné MP3 soubory nenalezeny v: {directory_path}")
            return 0

        print(f"📁 Nalezeno {len(mp3_files)} MP3 souborů")

        for file_path in mp3_files:
            try:
                self._process_mp3_file(str(file_path))
            except Exception as e:
                print(f"✗ Chyba při zpracování {file_path.name}: {e}")

        print(f"✓ Knihovna naskenována: {len(mp3_files)} skladeb")
        return len(mp3_files)

    def _process_mp3_file(self, file_path):
        """Zpracuje jeden MP3 soubor a vytvoří uzly"""
        # Načtení metadat
        try:
            audio = MP3(file_path, ID3=EasyID3)
            duration = int(audio.info.length)
            title = audio.get('title', [Path(file_path).stem])[0]
            artist_name = audio.get('artist', ['Unknown Artist'])[0]
            genre_name = audio.get('genre', ['Unknown'])[0]
        except:
            # Pokud metadata nejsou dostupná
            audio = MP3(file_path)
            duration = int(audio.info.length)
            title = Path(file_path).stem
            artist_name = "Unknown Artist"
            genre_name = "Unknown"

        # Vytvoření unikátních ID
        track_id = hashlib.md5(file_path.encode()).hexdigest()[:12]
        artist_id = hashlib.md5(artist_name.encode()).hexdigest()[:8]

        # Vytvoření uzlů a vztahů v Neo4j
        query = """
        // Vytvoření nebo nalezení Artist
        MERGE (a:Artist {artistId: $artist_id})
        ON CREATE SET a.name = $artist_name

        // Vytvoření nebo nalezení Genre
        MERGE (g:Genre {name: $genre_name})

        // Vytvoření Track (pokud neexistuje)
        MERGE (t:Track {trackId: $track_id})
        ON CREATE SET
            t.title = $title,
            t.duration = $duration,
            t.filePath = $file_path

        // Vytvoření vztahů
        MERGE (t)-[:IS_PERFORMED_BY]->(a)
        MERGE (t)-[:BELONGS_TO]->(g)
        """

        self.conn.query(query, {
            "track_id": track_id,
            "title": title,
            "duration": duration,
            "file_path": file_path,
            "artist_id": artist_id,
            "artist_name": artist_name,
            "genre_name": genre_name
        })

    def get_all_tracks(self):
        """Vrátí všechny skladby z databáze"""
        query = """
        MATCH (t:Track)-[:IS_PERFORMED_BY]->(a:Artist)
        OPTIONAL MATCH (t)-[:BELONGS_TO]->(g:Genre)
        RETURN t.trackId as trackId,
               t.title as title,
               t.duration as duration,
               t.filePath as filePath,
               a.name as artist,
               g.name as genre
        ORDER BY t.title
        """
        return self.conn.query(query)

In [ ]:
# === 4. DOPORUČOVACÍ SYSTÉMY ===
class MusicRecommender:
    def __init__(self, neo4j_conn):
        self.conn = neo4j_conn

    def collaborative_filtering(self, user_id, limit=10):
        """Kolaborativní filtrování - doporučení podle podobných uživatelů"""
        query = """
        // Najdi skladby, které poslouchal aktuální uživatel
        MATCH (u:User {userId: $user_id})-[l1:LISTENED_TO]->(t:Track)
        WITH u, collect(t) as user_tracks

        // Najdi podobné uživatele (kteří poslouchali stejné skladby)
        MATCH (other:User)-[l2:LISTENED_TO]->(t2:Track)
        WHERE other <> u AND t2 IN user_tracks
        WITH u, other, count(t2) as common_tracks, user_tracks
        WHERE common_tracks > 0
        ORDER BY common_tracks DESC
        LIMIT 10

        // Najdi skladby, které poslouchali podobní uživatelé, ale aktuální ne
        MATCH (other)-[:LISTENED_TO]->(rec:Track)
        WHERE NOT rec IN user_tracks
        WITH rec, count(DISTINCT other) as popularity
        ORDER BY popularity DESC
        LIMIT $limit

        MATCH (rec)-[:IS_PERFORMED_BY]->(a:Artist)
        OPTIONAL MATCH (rec)-[:BELONGS_TO]->(g:Genre)
        RETURN rec.trackId as trackId, rec.title as title,
               a.name as artist, g.name as genre, rec.filePath as filePath,
               popularity
        """
        return self.conn.query(query, {"user_id": user_id, "limit": limit})

    def content_based_filtering(self, user_id, limit=10):
        """Filtrování založené na obsahu - podle žánrů a umělců"""
        query = """
        // Najdi žánry a umělce, které uživatel poslouchá
        MATCH (u:User {userId: $user_id})-[:LISTENED_TO]->(t:Track)
        MATCH (t)-[:BELONGS_TO]->(g:Genre)
        MATCH (t)-[:IS_PERFORMED_BY]->(a:Artist)
        WITH u, collect(DISTINCT g) as user_genres, collect(DISTINCT a) as user_artists,
             collect(t) as listened_tracks

        // Najdi skladby ze stejných žánrů nebo od stejných umělců
        MATCH (rec:Track)-[:BELONGS_TO]->(g2:Genre)
        WHERE g2 IN user_genres AND NOT rec IN listened_tracks
        MATCH (rec)-[:IS_PERFORMED_BY]->(a2:Artist)
        WITH rec, a2,
             CASE WHEN a2 IN user_artists THEN 2 ELSE 1 END as score
        ORDER BY score DESC, rec.title
        LIMIT $limit

        OPTIONAL MATCH (rec)-[:BELONGS_TO]->(g:Genre)
        RETURN rec.trackId as trackId, rec.title as title,
               a2.name as artist, g.name as genre, rec.filePath as filePath,
               score
        """
        return self.conn.query(query, {"user_id": user_id, "limit": limit})

    # Hybridní doporučení s parametrem alpha
    def hybrid_recommendation(self, user_id, limit=10, alpha=0.6):
        query = """
        // Zjisti preference uživatele
        MATCH (u:User {userId: $user_id})-[:LISTENED_TO]->(t:Track)
        WITH u, collect(t) as listened_tracks

        MATCH (u)-[:LISTENED_TO]->(:Track)-[:BELONGS_TO]->(g:Genre)
        WITH u, listened_tracks, collect(DISTINCT g) as user_genres

        // Najdi podobné uživatele (peer group)
        MATCH (u)-[:LISTENED_TO]->(t_common:Track)<-[:LISTENED_TO]-(other:User)
        WITH u, listened_tracks, user_genres, collect(DISTINCT other) as peer_group

        // Hledej kandidáty (neslyšené skladby)
        MATCH (rec:Track)
        WHERE NOT rec IN listened_tracks

        // Výpočet dílčích skóre

        // Kolaborativní skóre (S_collab)
        OPTIONAL MATCH (rec)<-[:LISTENED_TO]-(peer)
        WHERE peer IN peer_group
        WITH rec, user_genres, count(peer) as raw_collab_score

        // Obsahové skóre (S_content)
        OPTIONAL MATCH (rec)-[:BELONGS_TO]->(rg:Genre)
        WITH rec, raw_collab_score,
             CASE WHEN rg IN user_genres THEN 1.0 ELSE 0.0 END as content_score

        // Normalizace a Finální výpočet
        WITH rec,
             (raw_collab_score * $alpha) + (content_score * (1.0 - $alpha)) as final_score

        WHERE final_score > 0
        ORDER BY final_score DESC
        LIMIT $limit

        // Vrácení výsledků
        MATCH (rec)-[:IS_PERFORMED_BY]->(a:Artist)
        OPTIONAL MATCH (rec)-[:BELONGS_TO]->(g:Genre)
        RETURN rec.trackId as trackId, rec.title as title,
               a.name as artist, g.name as genre, rec.filePath as filePath,
               final_score as score
        """
        return self.conn.query(query, {
            "user_id": user_id,
            "limit": limit,
            "alpha": alpha  # Předáváme alfu jako parametr
        })

    def record_listen(self, user_id, track_id, listen_duration, listen_date):
        """Zaznamenání poslechu skladby s časem poslechu"""
        query = """
        MATCH (u:User {userId: $user_id}), (t:Track {trackId: $track_id})
        MERGE (u)-[l:LISTENED_TO]->(t)
        ON CREATE SET l.listenDate = $listen_date, l.listenDuration = $listen_duration
        ON MATCH SET l.listenDate = $listen_date, l.listenDuration = l.listenDuration + $listen_duration
        """
        self.conn.query(query, {
            "user_id": user_id,
            "track_id": track_id,
            "listen_duration": listen_duration,
            "listen_date": listen_date
        })

    def add_fan_relationship(self, user_id, artist_id):
        """Přidání vazby IS_A_FAN_OF mezi userem a artistem"""
        query = """
        MATCH (u:User {userId: $user_id}), (a:Artist {artistId: $artist_id})
        MERGE (u)-[:IS_A_FAN_OF]->(a)
        """
        self.conn.query(query, {"user_id": user_id, "artist_id": artist_id})

In [ ]:
# === 5. HUDEBNÍ PŘEHRÁVAČ ===
class MusicPlayer:
    def __init__(self, recommender):
        pygame.mixer.init()
        self.recommender = recommender
        self.current_track_id = None
        self.current_artist_id = None
        self.is_playing = False
        self.current_user_id = None
        self.play_start_time = None

    def play(self, file_path, track_id, artist_id):
        """Přehrání skladby"""
        try:
            # Pokud hrála předchozí skladba, zaznamená se čas poslechu
            if self.current_track_id and self.play_start_time:
                self._record_listen_time()

            pygame.mixer.music.load(file_path)
            pygame.mixer.music.play()
            self.is_playing = True
            self.current_track_id = track_id
            self.current_artist_id = artist_id
            self.play_start_time = time.time()

            return True, f"▶ Přehrává se"
        except Exception as e:
            return False, f"✗ Chyba: {e}"

    def pause(self):
        if self.is_playing:
            pygame.mixer.music.pause()
            self.is_playing = False
            # Při pozastavení se nezaznamenává čas
            return "⏸ Pozastaveno"
        else:
            pygame.mixer.music.unpause()
            self.is_playing = True
            return "▶ Pokračuje"

    def stop(self):
        """Zastavení a zaznamenání času poslechu"""
        if self.current_track_id and self.play_start_time:
            self._record_listen_time()

        pygame.mixer.music.stop()
        self.is_playing = False
        self.current_track_id = None
        self.current_artist_id = None
        self.play_start_time = None
        return "⏹ Zastaveno"

    def _record_listen_time(self):
        """Interní metoda pro zaznamenání času poslechu"""
        if not self.current_user_id or not self.current_track_id:
            return

        listen_duration = int(time.time() - self.play_start_time)

        # Zaznamenání pouze pokud poslouchal alespoň 10 sekund
        if listen_duration >= 10:
            from datetime import datetime
            listen_date = datetime.now().isoformat()
            self.recommender.record_listen(
                self.current_user_id,
                self.current_track_id,
                listen_duration,
                listen_date
            )

    def add_artist_to_favorites(self):
        """Přidání aktuálního umělce do oblíbených"""
        if self.current_user_id and self.current_artist_id:
            self.recommender.add_fan_relationship(self.current_user_id, self.current_artist_id)
            return True, "✓ Umělec přidán do oblíbených"
        return False, "✗ Žádný umělec není načten"

In [ ]:
# === 6. UŽIVATELSKÉ ROZHRANÍ ===
class MusicAppUI:
    def __init__(self, music_dir):
        self.user_manager = UserManager(neo4j_conn)
        self.scanner = MusicLibraryScanner(neo4j_conn)
        self.recommender = MusicRecommender(neo4j_conn)
        self.player = MusicPlayer(self.recommender)
        self.music_dir = music_dir
        self.current_user = None

        # Naskenování knihovny
        print("📁 Skenování hudební knihovny...")
        self.scanner.scan_directory(music_dir)

        self.create_login_screen()

    def create_login_screen(self):
        """Přihlašovací obrazovka"""
        self.main_output = widgets.Output()

        username_input = widgets.Text(
            placeholder='Zadej uživatelské jméno',
            description='Username:',
            style={'description_width': '100px'}
        )

        login_btn = widgets.Button(description='🔑 Přihlásit se', button_style='primary')
        register_btn = widgets.Button(description='📝 Registrovat', button_style='success')
        status_output = widgets.Output()

        def on_login(b):
            with status_output:
                clear_output()
                user_id, msg = self.user_manager.login_user(username_input.value)
                if user_id:
                    print(f"✓ {msg}")
                    self.current_user = {"userId": user_id, "name": username_input.value}
                    self.player.current_user_id = user_id
                    time.sleep(1)
                    self.create_player_screen()
                else:
                    print(f"✗ {msg}")

        def on_register(b):
            with status_output:
                clear_output()
                user_id, msg = self.user_manager.register_user(username_input.value)
                if user_id:
                    print(f"✓ {msg}")
                    self.current_user = {"userId": user_id, "name": username_input.value}
                    self.player.current_user_id = user_id
                    time.sleep(1)
                    self.create_player_screen()
                else:
                    print(f"✗ {msg}")

        login_btn.on_click(on_login)
        register_btn.on_click(on_register)

        with self.main_output:
            clear_output()
            display(widgets.HTML("<h2>🎵 Hudební přehrávač - Přihlášení</h2>"))
            display(widgets.VBox([
                username_input,
                widgets.HBox([login_btn, register_btn]),
                status_output
            ]))

    def create_player_screen(self):
        """Hlavní obrazovka přehrávače"""
        with self.main_output:
            clear_output()

            # Header
            display(widgets.HTML(f"<h2>🎵 Vítej, {self.current_user['name']}!</h2>"))

            # Výběr skladeb
            tracks = self.scanner.get_all_tracks()
            track_list = widgets.Select(
                options=[(f"{t['title']} - {t['artist']} ({t.get('genre', 'Unknown')})", t)
                         for t in tracks],
                description='Skladby:',
                rows=10,
                layout=widgets.Layout(width='600px')
            )

            # Ovládací tlačítka
            play_btn = widgets.Button(description='▶ Přehrát', button_style='success')
            pause_btn = widgets.Button(description='⏸ Pauza', button_style='warning')
            stop_btn = widgets.Button(description='⏹ Stop', button_style='danger')
            favorite_btn = widgets.Button(description='⭐ Přidat umělce do oblíbených', button_style='info')

            # Doporučovací systém
            rec_type = widgets.Dropdown(
                options=['Kolaborativní filtrování', 'Obsahové filtrování', 'Hybridní doporučení'],
                description='Doporučení:',
                style={'description_width': '120px'}
            )
            rec_btn = widgets.Button(description='🎯 Doporuč skladby', button_style='info')
            rec_output = widgets.Output()

            player_status = widgets.Output()
            favorite_status = widgets.Output()

            def on_play(b):
                with player_status:
                    clear_output()
                    if track_list.value:
                        track = track_list.value
                        # Najdi artistId pro aktuální skladbu
                        artist_query = """
                        MATCH (t:Track {trackId: $track_id})-[:IS_PERFORMED_BY]->(a:Artist)
                        RETURN a.artistId as artistId
                        """
                        artist_result = neo4j_conn.query(artist_query, {"track_id": track['trackId']})
                        artist_id = artist_result[0]['artistId'] if artist_result else None

                        success, msg = self.player.play(track['filePath'], track['trackId'], artist_id)
                        if success:
                            print(f"{msg}: {track['title']} - {track['artist']}")
                        else:
                            print(msg)

            def on_pause(b):
                with player_status:
                    clear_output()
                    print(self.player.pause())

            def on_stop(b):
                with player_status:
                    clear_output()
                    print(self.player.stop())

            def on_favorite(b):
                with favorite_status:
                    clear_output()
                    success, msg = self.player.add_artist_to_favorites()
                    print(msg)

            def on_recommend(b):
                with rec_output:
                    clear_output()
                    print("🔍 Hledám doporučení...")

                    if rec_type.value == 'Kolaborativní filtrování':
                        recs = self.recommender.collaborative_filtering(self.current_user['userId'])
                    elif rec_type.value == 'Obsahové filtrování':
                        recs = self.recommender.content_based_filtering(self.current_user['userId'])
                    else:
                        recs = self.recommender.hybrid_recommendation(self.current_user['userId'])

                    clear_output()
                    if recs:
                        print(f"✓ Nalezeno {len(recs)} doporučení:\n")
                        for i, r in enumerate(recs, 1):
                            print(f"{i}. {r['title']} - {r['artist']} ({r.get('genre', 'Unknown')})")
                    else:
                        print("✗ Zatím nemám dostatek dat pro doporučení. Poslouchej více hudby!")

            play_btn.on_click(on_play)
            pause_btn.on_click(on_pause)
            stop_btn.on_click(on_stop)
            favorite_btn.on_click(on_favorite)
            rec_btn.on_click(on_recommend)

            # Layout
            display(track_list)
            display(widgets.HBox([play_btn, pause_btn, stop_btn, favorite_btn]))
            display(favorite_status)
            display(player_status)
            display(widgets.HTML("<hr>"))
            display(widgets.HBox([rec_type, rec_btn]))
            display(rec_output)

In [ ]:
# === 7. SPUŠTĚNÍ APLIKACE ===
def start_app(music_directory):
    """Spuštění aplikace"""
    if not os.path.exists(music_directory):
        print(f"✗ Složka {music_directory} neexistuje!")
        return

    app = MusicAppUI(music_directory)
    display(app.main_output)

# POUŽITÍ:
start_app(r"C:\Users\marti\Desktop\Škola\Neo4j mp3s")